In [1]:
# import libraries

import folium
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [2]:
# create a dataframe from the csv and then print it

df = pd.read_csv('churn.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
# get a summary of the dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
# data preprocessing

df['IsActiveMember'] = df['IsActiveMember'].replace(1,'Active Member')
df['IsActiveMember'] = df['IsActiveMember'].replace(0,'Non Active Member')
df['Exited'] = df['Exited'].replace(1,'Yes')
df['Exited'] = df['Exited'].replace(0,'No')
df['HasCrCard'] = df['HasCrCard'].replace(1,'Yes')
df['HasCrCard'] = df['HasCrCard'].replace(0,'No')
df = df.drop('RowNumber', axis = 1)

In [5]:
# df

In [6]:
# create some parameters for the graphs

age_min = int(df['Age'].min())
age_max = int(df['Age'].max())

a =[
    {'label':'Credit Score','value':'CreditScore'},
    {'label':'Tenure','value':'Tenure'},
    {'label':'Balance','value':'Balance'},
    {'label':'No. of products','value':'NumOfProducts'},
    {'label':'Credit Card','value':'HasCrCard'},
    {'label':'Is Active','value':'IsActiveMember'},
    {'label':'Estimated Salary','value':'EstimatedSalary'},
    {'label':'Exited','value':'Exited'},
   ]

b =[
    {'label':'Credit Score','value':'CreditScore'},
    {'label':'Tenure','value':'Tenure'},
    {'label':'Balance','value':'Balance'},
    {'label':'No. of products','value':'NumOfProducts'},
    {'label':'Credit Card','value':'HasCrCard'},
    {'label':'Estimated Salary','value':'EstimatedSalary'},
    {'label':'Is Active','value':'IsActiveMember'}
   ]

In [7]:
# initiate the app

app = JupyterDash()


# geoJson with world countries boundaries to combine with our Data Frame,
# since we do not have coordinates but only country names

state_geo = (
    "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/world-countries.json"
)


# render map with sliced dataframe

def map_from_slice(dataframe, category):

    base_map = folium.Map(location=(46.2022, 1.2644), tiles= 'cartodbpositron', zoom_start= 4)
    
    folium.Choropleth(
        geo_data= state_geo, 
        data= dataframe, 
        columns= ['Geography', category],
        key_on='feature.properties.name',
        fill_opacity=0.5,
        fill_color= 'PuBuGn',
        nan_fill_color= 'Lightgrey',
        nan_fill_opacity=1,
        legend_name= category,
        highlight = True
    ).add_to(base_map)
    
    return base_map


# set app layout

app.layout = html.Div(children =[
    
    # intro with fade in and out
    html.Div(className='splash', children=[
        html.H1('Welcome', className='fade-in'),
        html.Br(),
        html.P('A graphical reprasentation of churn rate in', className='fade-in'),
        html.Ul(className= 'fade-in', children=[
            html.Li(className= 'fade-in', children=['Germany']),
            html.Li(className= 'fade-in', children=['France']),
            html.Li(className= 'fade-in', children=['Spain'])
        ]),
        html.Br(),
        html.P('Code set up by:', className='fade-in'),
        html.Ul(className= 'fade-in', children=[
            html.Li(className= 'fade-in', children=['Thomas Rallis']),
            html.Li(className= 'fade-in', children=['Panagiotis Gkrigkas'])
        ])
    ]),
    
    # add each dropdown in a column of 25% width
    html.Div(className='row', children=[
        # dropdown for gender selection
        html.Div(className='column25', children=[
            html.H5('Gender'),
            dcc.Dropdown(
                id='Gender-dropdown',
                value='All',
                style={'display':'inline-block', 'width':'100%'},
                options= [{'label': 'All','value': 'All'}]+[{'label': gender,'value': gender} for gender in df['Gender'].unique()]
            )
        ]),
        
        html.Div(className='column25', children=[
            # dropdown interaction with map
            html.H5('Category'),
            dcc.Dropdown(
                id='Category',
                value='EstimatedSalary',
                style={'display':'inline-block', 'width':'100%'},
                options= a
            )
        ]),
        
        html.Div(className='column25', children=[
            # dropdown interaction with scatter plot
            html.H5('Scatterplot'),
            dcc.Dropdown(
                id='Variable 1',
                value='CreditScore',
                style={'display':'inline-block', 'width':'100%'},
                options= a
            )
        ]),
        
        html.Div(className='column25', children=[
            # dropdown interaction with histogram
            html.H5('Histogram'),
            dcc.Dropdown(
                id='Variable 2',
                value='IsActiveMember',
                style={'display':'inline-block', 'width':'100%'},
                options= b
            )
        ])
    ]),
    
    # age slider
    html.Div(className='row2', children=[
        html.Div(className='column70', children=[
            html.H5('Age'),
            dcc.RangeSlider(
                id = 'age-slider',
                min = age_min ,
                max = age_max ,
                marks={age_min:str(age_min), 20:'20', 30: '30' , 40: '40', 50: '50',
                       60: '60', 70: '70', 80: '80', 90:'90', age_max:str(age_max)},
                value=[25,55]
            )
        ])
    ]),
    
    
    # map 
    html.Div(className= 'column' ,children=[
        html.Div(className= 'card' ,children=[
            html.H3('Map'),
            html.Iframe(id = 'map', width='100%', height='120%' )
        ])
    ]),
    
    
    # scatterplot
    html.Div(className= 'column', children=[
        html.Div(className= 'card', children=[
            html.H3('Scatterplot'),
            dcc.Graph(id = 'scatter', figure = {}),
        ])
    ]),
    
    # boxplot & histogramm
    html.Div(className='row', children=[
        html.Div(className='column', children=[
            html.Div(className= 'card', children=[
                html.H3('Box Plot'),
                dcc.Graph(id = 'box', figure = {})
            ])
        ]),
        
    html.Div(className= 'column', children=[
            html.Div(className='card', children=[
                html.H3('Histogram'),
                dcc.Graph(id='hist', figure= {})
            ])
        ])
    ])
])
    
# create a callback to connect slider and dropdowns with the graph
    
@app.callback(
    [
        Output('map', 'srcDoc'),
        Output('scatter','figure'),
        Output('box', 'figure'),
        Output('hist', 'figure')
    ],
    [
        Input('age-slider', 'value'),
        Input('Gender-dropdown', 'value'),
        Input('Category', 'value'),
        Input('Variable 1', 'value'),
        Input('Variable 2', 'value'),
    ]
    )

# create a fuction that updates the map with the values on slider and dropdown menus

def update_graph(slider_range, gender, cat , var1 , var2):
    slider_min = slider_range[0]
    slider_max = slider_range[1]
    
    if gender == 'All':
        df_slice = df[(df['Age'] >= slider_min) & (df['Age'] <= slider_max)]
    else:
        df_slice = df[(df['Age'] >= slider_min) & (df['Age'] <= slider_max) & (df['Gender']==gender)]
    
    fig1 = px.scatter(df_slice , x = var1 , y = cat , size = 'NumOfProducts' , color = 'CreditScore')
    
    fig2 = px.box(df_slice, x ='Tenure', y= 'EstimatedSalary', color= 'Exited' ,
                  labels = {'EstimatedSalary' : 'Estimated Salary'})
    
    fig3 = px.histogram(df_slice , x = var2 , color = 'Exited')
    
    base_map = map_from_slice(df_slice, cat)
    
    return base_map._repr_html_(), fig1, fig2, fig3

app.run_server(host="localhost", port=8084)

Dash is running on http://localhost:8084/

Dash app running on http://localhost:8084/
